<a href="https://colab.research.google.com/github/gustheman/ml_for_audio/blob/main/recording_audio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quick sample to record audio on colab

In [ ]:
!pip -q install pydub

In [ ]:
from IPython.display import Audio, Javascript
from base64 import b64decode
import matplotlib.pyplot as plt
from scipy.io import wavfile
from pydub import AudioSegment

In [ ]:
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=5):
  try:
    from google.colab import output
  except ImportError:
    print('No possible to import output from google.colab')
    return ''
  else:
    print('Recording')
    display(Javascript(RECORD))
    s = output.eval_js('record(%d)' % (sec*1000))
    fname = 'recorded_audio.wav'
    print('Saving to', fname)
    b = b64decode(s.split(',')[1])
    with open(fname, 'wb') as f:
      f.write(b)
    return fname

In [ ]:
recorded_file_name = record(5)

In [ ]:
# tricky to fix the audio just recorded and change the sample_rate
def resample_audio(audio_file_name, output_sample_rate=16000):
  audio = AudioSegment.from_file(audio_file_name)
  audio = audio.set_frame_rate(output_sample_rate).set_channels(1)
  audio.export(audio_file_name, format="wav")

resample_audio(recorded_file_name)

In [ ]:
print(f'loading file {recorded_file_name}')
sample_rate, audio_samples = wavfile.read(recorded_file_name, 'rb')

print(f'sample_rate: {sample_rate}')
print(f'duration: {len(audio_samples)/sample_rate}')

Audio(recorded_file_name)

In [ ]:
#showing the waveform
plt.plot(audio_samples)